In [16]:
from PIL import Image

width = 100
height = 100

# l - znaczy ze w skali szarosci
image = Image.new("L", (width, height), color=255)

# dodanie czarnego paska
for x in range(width):
    for y in range(height):
        if width // 3 <= x < 2 * (width // 3):
            image.putpixel((x, y), 0)  

image.save("sample_image.png")
image.show()

In [17]:
from PIL import ImageDraw

width = 100
height = 100

image = Image.new("L", (width, height), color=255)

draw = ImageDraw.Draw(image)
circle_radius = 30
circle_center = (width // 2, height // 2)
draw.ellipse((circle_center[0] - circle_radius, circle_center[1] - circle_radius,
              circle_center[0] + circle_radius, circle_center[1] + circle_radius),
              fill=0)


image.save("circle_image.png")
image.show()


In [18]:
from PIL import Image
import random

def encrypt_image(image_path):
    original_image = Image.open(image_path)
    
    share1 = Image.new("L", (original_image.width*2, original_image.height))
    share2 = Image.new("L", (original_image.width*2, original_image.height))
    
    
    for x in range(original_image.width):
        for y in range(original_image.height):
            pixel = original_image.getpixel((x, y))
            match pixel:
                case 255:
                    # jesli bialy to oba share sa takie same
                    if random.random() > 0.5:
                        random_share1 = [0, 255]
                        random_share2 = [0, 255]
                    else:
                        random_share1 = [255, 0]
                        random_share2 = [255, 0] 
                case 0:
                    # jesli originalny jest czarny to oba share sa inne
                    if random.random() > 0.5:
                        random_share1 = [0, 255]
                        random_share2 = [255, 0]
                    else:
                        random_share1 = [255, 0]
                        random_share2 = [0, 255] 

            share1.putpixel((x*2, y), random_share1[0])
            share1.putpixel((x*2 + 1, y), random_share1[1])
            share2.putpixel((x*2, y), random_share2[0])
            share2.putpixel((x*2 + 1, y), random_share2[1])
        
    return share1, share2

def decrypt_image(share1, share2):
    # powrot do starego shape
    decrypted_image = Image.new("L", (share1.width // 2, share1.height))
    
    
    for x in range(0, share1.width, 2):
        for y in range(share1.height):
            pixel11 = share1.getpixel((x, y))
            pixel12 = share2.getpixel((x, y))
            pixel21 = share1.getpixel((x + 1, y))
            pixel22 = share2.getpixel((x + 1, y))
            # laczenie
            # jeśli jakies dwa nakladajace sie piksele sa biale to bialy
            decrypted_pixel = 255 if (pixel11 + pixel12) > 255 or (pixel21 + pixel22) > 255 else 0
            decrypted_image.putpixel((x//2, y), decrypted_pixel)
    
    return decrypted_image

In [19]:
image_path = "circle_image.png"

share1, share2 = encrypt_image(image_path)

share1.save("share1.png")
share2.save("share2.png")

decrypted_image = decrypt_image(share1, share2)
decrypted_image.save("decrypted_image.png")
